### Data Cleaning

Cleaning up the dataset for further analysis.

The assumption is that: 
<br>`Total Volume` = `Small Hass` + `Large Hass` + `XL Hass`  + `Small Bags` + `Large Bags` + `XLarge Bags` which are all in thousands.
<br>`Total Bags` = `Small Bags` + `Large Bags` + `XLarge Bags` wich are also in thousands.
<br>`AveragePrice` is in contemporary dollars.


Because `AveragePrice` could be an interesting variable to explore we'll adjust it to 2017 dollars using the [Seasonally Adjusted Consumer Price Index (CPI) from the Bureau of Labor Statistics](https://www.bls.gov/cpi/seasonal-adjustment/home.htm).

In [2]:
import pandas as pd

### Original dataset

In [3]:
df = pd.read_csv('https://query.data.world/s/fs2xufwllyqmhuvjkudsmvbr5djiag')

#### CPI for Fresh Fruit and Vegetables

In [4]:
cpi = pd.read_excel("https://www.bls.gov/cpi/tables/seasonal-adjustment/revised-seasonally-adjusted-indexes-2017.xlsx")

Since we are looking at avocado prices we'll use the Seasonal Factor for __Fresh fruits and vegetables__

In [5]:
food = cpi[(cpi['TITLE'] == 'Fresh fruits and vegetables') & (cpi['DATA_TYPE'] == 'SEASONAL FACTOR')].copy()

In [6]:
food.head()

,ITEM,TITLE,seriesid,DATA_TYPE,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC
530,SAF1131,Fresh fruits and vegetables,CUSR0000SAF1131,SEASONAL FACTOR,2013,101.850,101.171,100.256,100.054,100.339,99.042,98.364,98.597,99.516,100.303,100.143,100.603
531,SAF1131,Fresh fruits and vegetables,CUSR0000SAF1131,SEASONAL FACTOR,2014,101.856,101.059,100.042,99.856,100.296,98.897,98.400,98.708,99.672,100.669,100.372,100.487
532,SAF1131,Fresh fruits and vegetables,CUSR0000SAF1131,SEASONAL FACTOR,2015,101.777,100.928,99.976,99.779,100.236,98.830,98.417,98.746,99.699,100.861,100.487,100.439
533,SAF1131,Fresh fruits and vegetables,CUSR0000SAF1131,SEASONAL FACTOR,2016,101.718,100.895,99.907,99.803,100.185,98.737,98.389,98.795,99.812,100.993,100.540,100.410
534,SAF1131,Fresh fruits and vegetables,CUSR0000SAF1131,SEASONAL FACTOR,2017,101.593,100.808,99.836,99.863,100.128,98.735,98.394,98.828,99.850,101.009,100.519,100.416


In [7]:
food.drop_duplicates(subset= ['YEAR'], inplace=True)

food_cpi = food.melt(id_vars='YEAR', value_vars=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP','OCT', 'NOV','DEC'])

#### Cleaning original dataset
We'll focus on 2015-2017 since we have those years complete.

In [8]:
df['Date'] = pd.to_datetime(df['Date'], format = "%d/%m/%Y")

In [9]:
df = df[df['Date'] < '2018-01-01'].copy()

In [10]:
df['YEAR-MONTH'] = df['Year'].astype(str) + '-' + df['Date'].dt.month.astype(str)

#### Prepping `food_cpi`

In [11]:
months = {
    "JAN": "1",
    "FEB": "2",
    "MAR": "3",
    "APR": "4",
    "MAY": "5",
    "JUN": "6",
    "JUL": "7",
    "AUG": "8",
    "SEP": "9",
    "OCT": "10",
    "NOV": "11",
    "DEC": "12",
}

food_cpi['MONTH'] = food_cpi['variable'].map(months)

food_cpi['YEAR-MONTH'] = food_cpi['YEAR'].astype(str) + '-' + food_cpi['MONTH']


food_cpi['CPI'] = food_cpi['value'] / 100

food_cpi = food_cpi[['YEAR-MONTH', 'CPI']].copy()

In [12]:
working_df = pd.merge(df, food_cpi,)

working_df.head()

,Date,AveragePrice,Total Volume,Small Hass,Large Hass,XL Hass,Total Bags,Small Bags,Large Bags,XLarge Bags,Type,Year,Region,YEAR-MONTH,CPI
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany,2015-12,1.00439
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany,2015-12,1.00439
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany,2015-12,1.00439
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany,2015-12,1.00439
4,2015-12-27,0.99,386100.49,292097.36,27350.92,297.90,66354.31,48605.95,17748.36,0.0,conventional,2015,Atlanta,2015-12,1.00439


In [13]:
working_df['SeasonallyAdjusted_AveragePrice'] = (working_df['AveragePrice'] * working_df['CPI']).round(2)

In [14]:
working_df.head()

,Date,AveragePrice,Total Volume,Small Hass,Large Hass,XL Hass,Total Bags,Small Bags,Large Bags,XLarge Bags,Type,Year,Region,YEAR-MONTH,CPI,SeasonallyAdjusted_AveragePrice
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany,2015-12,1.00439,1.34
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany,2015-12,1.00439,1.36
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany,2015-12,1.00439,0.93
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany,2015-12,1.00439,1.08
4,2015-12-27,0.99,386100.49,292097.36,27350.92,297.90,66354.31,48605.95,17748.36,0.0,conventional,2015,Atlanta,2015-12,1.00439,0.99


## This is a good checkpoint

In [15]:
cols_of_interest = [
    'Date', 
    'Year', 
    'Type', 
    'Region', 
    'SeasonallyAdjusted_AveragePrice',
    'AveragePrice', 
    'Total Volume', 
    'Small Hass', 
    'Large Hass',
    'XL Hass',  
    'Small Bags', 
    'Large Bags', 
    'XLarge Bags',
    'Total Bags',
]

In [32]:
working_df[cols_of_interest].to_csv("../data/processed/working-szn_adjusted_prices.csv", index = False,)

In [20]:
# One of just CA
ca_df = working_df[working_df['Region'] == 'California'].copy()

ca_df[cols_of_interest].to_csv("../data/processed/working-CA_szn_adjusted_prices.csv", index = False,)

#### Adding percent change

In [21]:
organic_df = ca_df[ca_df['Type'] == 'organic'].copy()
conventional_df = ca_df[ca_df['Type'] == 'conventional'].copy()

In [22]:
weighted_mean = ((ca_df['SeasonallyAdjusted_AveragePrice'] * ca_df['Total Volume']).sum()) / ca_df['Total Volume'].sum()

organic_mean = ((organic_df['SeasonallyAdjusted_AveragePrice'] * organic_df['Total Volume']).sum()) / organic_df['Total Volume'].sum()

conventional_mean = ((conventional_df['SeasonallyAdjusted_AveragePrice'] * conventional_df['Total Volume']).sum()) / conventional_df['Total Volume'].sum()

In [23]:
ca_df.loc[ca_df['Type'] == 'conventional', 'percent_change'] = ca_df['SeasonallyAdjusted_AveragePrice'] / conventional_mean

ca_df.loc[ca_df['Type'] == 'organic', 'percent_change'] = ca_df['SeasonallyAdjusted_AveragePrice'] / organic_mean

In [29]:
cols_of_interest = [
    'Date', 
    'Year', 
    'Type', 
    'Region', 
    'percent_change',
    'SeasonallyAdjusted_AveragePrice',
    'AveragePrice', 
    'Total Volume', 
    'Small Hass', 
    'Large Hass',
    'XL Hass',  
    'Small Bags', 
    'Large Bags', 
    'XLarge Bags',
    'Total Bags',
]

ca_df[cols_of_interest].to_csv("../data/processed/working-CA_pct_changes.csv", index = False,)